## Adding another collocation point: Montalban (x = 0)

In [3]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
# from tensorflow import keras
import tensorflow as tf

# %load_ext tensorboard
# import tensorboard
import os

In [4]:
from datetime import datetime
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

In [5]:
df = pd.read_csv("compiled_data_2016_2017.csv")
df.head()

,Unnamed: 0,index,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,Waterlevel_Montalban,Discharge_Sto_Nino,Discharge_San_Jose,Cross_Section_Sto_Nino,Cross_Section_Montalban,Velocity_Sto_Nino,Velocity_Montalban,datetime,t,x
0,0,0,0,1,2,0,0,12.18,21.03,21.033407,14.842428,803.88,630.9,0.026165,0.023526,2016-01-01 00:00:00,0.0,14420
1,1,1,0,1,1,1,0,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 01:00:00,3600.0,14420
2,2,2,1,1,1,0,1,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 02:00:00,7200.0,14420
3,3,3,0,0,0,1,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 03:00:00,10800.0,14420
4,4,4,1,1,1,0,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 04:00:00,14400.0,14420


In [6]:
print(df.head())

   Unnamed: 0  index  Rainfall_Aries  Rainfall_Boso  Rainfall_Campana  \
0           0      0               0              1                 2   
1           1      1               0              1                 1   
2           2      2               1              1                 1   
3           3      3               0              0                 0   
4           4      4               1              1                 1   

   Rainfall_Nangka  Rainfall_Oro  Waterlevel_Sto_Nino  Waterlevel_Montalban  \
0                0             0                12.18                 21.03   
1                1             0                12.19                 21.03   
2                0             1                12.19                 21.03   
3                1             0                12.20                 21.03   
4                0             0                12.20                 21.03   

   Discharge_Sto_Nino  Discharge_San_Jose  Cross_Section_Sto_Nino  \
0           21.03

In [7]:
df1 = pd.DataFrame(columns=['x', 't', 'Discharge', 'friction_coeff', 'slope', 'water_velocity', 'water_level'])

In [8]:
df1

,x,t,Discharge,friction_coeff,slope,water_velocity,water_level


In [14]:
import pandas as pd

def transform_dataframe(df):
    """
    Transform the input DataFrame to create two rows (Montalban and Sto. Nino) for each time step
    with specified columns.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame containing the original data
    
    Returns:
    pd.DataFrame: Transformed DataFrame with the specified structure
    """
    # Initialize lists to store the data
    rows = []
    
    # Process each row in the original DataFrame
    for _, row in df.iterrows():
        # Sto. Nino row (x = 14420)
        sto_nino = {
            'x': 14420,
            't': row['t'],
            'Discharge': row['Discharge_Sto_Nino'],
            'friction_coeff': 0.033,  # Assuming this is constant, adjust if needed
            'slope': 1/1500,        # Assuming this is constant, adjust if needed
            'water_velocity': row['Velocity_Sto_Nino'],
            'water_level': row['Waterlevel_Sto_Nino']
        }
        
        # Montalban row (x = 0)
        montalban = {
            'x': 0,
            't': row['t'],
            'Discharge': row['Discharge_San_Jose'],
            'friction_coeff': 0.033,  # Assuming this is constant, adjust if needed
            'slope': 1/1500,        # Assuming this is constant, adjust if needed
            'water_velocity': row['Velocity_Montalban'],
            'water_level': row['Waterlevel_Montalban']
        }
        
        # Add both rows to the list
        rows.extend([montalban, sto_nino])
    
    # Create new DataFrame
    new_df = pd.DataFrame(rows)
    
    # Sort by time and x coordinate
    new_df = new_df.sort_values(['t', 'x']).reset_index(drop=True)
    
    return new_df

In [15]:
df1 = transform_dataframe(df)

In [16]:
df1

,x,t,Discharge,friction_coeff,slope,water_velocity,water_level
0,0,0.0,14.842428,0.033,0.000667,0.023526,21.03
1,14420,0.0,21.033407,0.033,0.000667,0.026165,12.18
2,0,3600.0,14.842428,0.033,0.000667,0.023526,21.03
3,14420,3600.0,21.280072,0.033,0.000667,0.026450,12.19
4,0,7200.0,14.842428,0.033,0.000667,0.023526,21.03
...,...,...,...,...,...,...,...
35035,14420,63147600.0,28.244204,0.033,0.000667,0.034401,12.44
35036,0,63151200.0,17.224575,0.033,0.000667,0.027108,21.18
35037,14420,63151200.0,28.244204,0.033,0.000667,0.034401,12.44
35038,0,63154800.0,17.224575,0.033,0.000667,0.027108,21.18


In [20]:
n = len(df1)
split = int(n*0.50)

train_df = df1[:split]
train_df.tail()

,x,t,Discharge,friction_coeff,slope,water_velocity,water_level
17515,14420,31611600.0,31.158059,0.033,0.000667,0.037677,12.53
17516,0,31615200.0,19.589424,0.033,0.000667,0.030642,21.31
17517,14420,31615200.0,31.158059,0.033,0.000667,0.037677,12.53
17518,0,31618800.0,19.980443,0.033,0.000667,0.031224,21.33
17519,14420,31618800.0,31.158059,0.033,0.000667,0.037677,12.53


In [22]:
val_df = df1[split:]
val_df.head()

,x,t,Discharge,friction_coeff,slope,water_velocity,water_level
17520,0,31622400.0,21.199990,0.033,0.000667,0.033037,21.39
17521,14420,31622400.0,31.158059,0.033,0.000667,0.037677,12.53
17522,0,31626000.0,21.199990,0.033,0.000667,0.033037,21.39
17523,14420,31626000.0,31.158059,0.033,0.000667,0.037677,12.53
17524,0,31629600.0,21.199990,0.033,0.000667,0.033037,21.39


In [23]:
X_train = np.array(train_df[['x','t', 'Discharge', 'friction_coeff', 'slope']].values.tolist())
X_val = np.array(val_df[['x','t', 'Discharge', 'friction_coeff', 'slope']].values.tolist())

In [25]:
Y_train = np.array(train_df[['water_velocity','water_level']].values.tolist())
Y_val = np.array(val_df[['water_velocity','water_level']].values.tolist())

In [27]:
epochs = 15
best_val_loss = np.inf
best_model = -1 
results = pd.DataFrame(columns=['n1','n2','n3', 'epochs', 'reg',
                               'val_r2', 'val_nse', 'val_mse', 'val_mae', 'val_mape'])
layer_1_neurons = [16,32,64]   ######## Check the number of neurons ########
layer_2_neurons = [16,32,64]  ######## Check the number of neurons ########
layer_3_neurons = [16,32,64]  ######## Check the number of neurons ########
reg_consts = [0]

In [28]:
def r_square(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.square(K.sum(xm * ym))
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = (x_square_sum * y_square_sum) + K.epsilon()
    
    r = r_num / r_den
    return r

def NSE(y_true, y_pred):

    y_pred = K.flatten(y_pred)
    y_true = K.flatten(y_true)

    
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

#PDE Loss Function
def custom_loss(grads_inputs):
    du_dx, du_dt, dh_dx, fric_coeff, slope = grads_inputs[:,0], grads_inputs[:,1], grads_inputs[:,2], grads_inputs[:,3], grads_inputs[:,4]
    g = K.constant(9.8)
    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
    def loss(y_true,y_pred):
        loss_saint_venant = du_dt + y_pred[:,0] * du_dx + g*dh_dx + g*slope + g*K.square(fric_coeff) * K.square(y_true[:,0])/(K.pow(y_true[:,1], 4/3) + K.epsilon())
        l = K.mean(K.square(loss_saint_venant))

        return 2*l+ K.sum(K.mean(K.square(y_pred - y_true), axis=0))
   
    # Return a function
    return loss

In [32]:
for reg in reg_consts:
    for n1 in layer_1_neurons:
        for n2 in layer_2_neurons:
            for n3 in layer_3_neurons:
                print(n1,n2,n3,reg)
                K.clear_session()
                model = Sequential()
                model.add(Dense(n1, activation = 'relu', kernel_regularizer=l2(reg),input_shape = (5,)))
                model.add(Dense(n2, activation = 'relu', kernel_regularizer=l2(reg)))
                model.add(Dense(n3, activation = 'relu', kernel_regularizer=l2(reg)))
                model.add(Dense(2))
                grads_u = K.gradients(model.output[:,0], model.input)[0]
                grads_h = K.gradients(model.output[:,1], model.input)[0]
                du_dx, du_dt, dh_dx = grads_u[:,0],grads_u[:,1],grads_h[:,0]
                calc_grads_inputs = K.stack((du_dx, du_dt, dh_dx, model.input[:,3],model.input[:,4]), axis=1)
                
                model.compile(optimizer = 'adam', loss = [custom_loss(calc_grads_inputs)], metrics=['mape', 'mae', 'mse',NSE, r_square])
                
                early_stopping_monitor = EarlyStopping(patience = 2, verbose=False)
                history = model.fit(X_train,Y_train, epochs=epochs, batch_size=128, validation_data=(X_val,Y_val), callbacks=[early_stopping_monitor])
                
                # Saving results
                val_loss = history.history['val_loss'][-1]
                val_mae = history.history['val_mae'][-1]
                val_mse = history.history['val_mse'][-1]
                val_mape = history.history['val_mape'][-1]
                val_nse = history.history['val_NSE'][-1]
                val_r_square = history.history['val_r_square'][-1]
                
                # Create a new row as a DataFrame
                new_row = pd.DataFrame({
                    'n1': [n1],
                    'n2': [n2],
                    'n3': [n3], 
                    'epochs': [len(history.history['val_loss'])],
                    'reg': [reg],
                    'val_r2': [val_r_square], 
                    'val_nse': [val_nse], 
                    'val_mse': [val_mse], 
                    'val_loss': [val_loss],
                    'val_mae': [val_mae], 
                    'val_mape': [val_mape]
                })
                
                # Concatenate the new row with the existing results
                results = pd.concat([results, new_row], ignore_index=True)
                
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_model = model
                    best_n1 = n1
                    best_n2 = n2
                    best_n3 = n3
                    best_reg = reg
                    best_history = history

16 16 16 0
Train on 17520 samples, validate on 17520 samples
Epoch 1/15
17520/17520 [==============================] - 0s 27us/sample - loss: 452546016301.4459 - mape: 2357137920.0000 - mae: 203473.8594 - mse: 226208120832.0000 - NSE: -2851979008.0000 - r_square: 0.0070 - val_loss: 13096181.8767 - val_mape: 5101648.0000 - val_mae: 2493.5986 - val_mse: 6548043.5000 - val_NSE: -81778.8281 - val_r_square: 0.0250
Epoch 2/15
17520/17520 [==============================] - 0s 13us/sample - loss: 153916.6286 - mape: 1090006.3750 - mae: 154.9755 - mse: 76957.5391 - NSE: -970.9797 - r_square: 0.5279 - val_loss: 28416.0131 - val_mape: 59535.3672 - val_mae: 85.5782 - val_mse: 14207.9648 - val_NSE: -175.9839 - val_r_square: 0.7677
Epoch 3/15
17520/17520 [==============================] - 0s 13us/sample - loss: 9269.6211 - mape: 421945.4062 - mae: 49.2821 - mse: 4634.6982 - NSE: -57.5625 - r_square: 0.7966 - val_loss: 26216.6643 - val_mape: 44772.1562 - val_mae: 78.9353 - val_mse: 13108.3018 - val_N

In [33]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                384       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 16)                1040      
                                                                 
 dense_3 (Dense)             (None, 2)                 34        
                                                                 
Total params: 5618 (21.95 KB)
Trainable params: 5618 (21.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
best_model.history.history

{'loss': [320069192284.40173,
  374.35093782329125,
  330.7873224406482,
  303.35574407708157,
  302.3889713931846,
  301.3909462009935,
  300.9961346351937,
  300.99429558183505,
  300.8017989206532,
  301.10431705283247,
  300.6592421440229,
  301.4959329126088,
  304.2371867976777,
  303.80969455666735,
  301.5053435303849],
 'mape': [214473040.0,
  719.8915405273438,
  525.089111328125,
  485.9676513671875,
  478.701416015625,
  456.8942565917969,
  448.46173095703125,
  445.55999755859375,
  444.3213806152344,
  444.9088134765625,
  439.9854431152344,
  453.2740173339844,
  491.2711181640625,
  472.48345947265625,
  455.5790100097656],
 'mae': [94441.140625,
  8.530033111572266,
  8.445798873901367,
  8.41581916809082,
  8.409895896911621,
  8.402763366699219,
  8.400757789611816,
  8.399516105651855,
  8.399694442749023,
  8.398279190063477,
  8.396941184997559,
  8.401300430297852,
  8.413518905639648,
  8.406991004943848,
  8.40233325958252],
 'mse': [158416076800.0,
  187.1259

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
'''
def normalize_data(X_train, X_val, Y_train, Y_val):
    """Normalize input and output data"""
    X_mean = np.mean(X_train, axis=0)
    X_std = np.std(X_train, axis=0)
    Y_mean = np.mean(Y_train, axis=0)
    Y_std = np.std(Y_train, axis=0)
    
    X_train_norm = (X_train - X_mean) / (X_std + 1e-8)
    X_val_norm = (X_val - X_mean) / (X_std + 1e-8)
    Y_train_norm = (Y_train - Y_mean) / (Y_std + 1e-8)
    Y_val_norm = (Y_val - Y_mean) / (Y_std + 1e-8)
    
    return (X_train_norm, X_val_norm, Y_train_norm, Y_val_norm, 
            X_mean, X_std, Y_mean, Y_std)
'''
def custom_loss(grads_inputs, physics_weight=1.0):
    """Modified loss function with adjustable physics weight and improved numerical stability"""
    du_dx, du_dt, dh_dx, fric_coeff, slope = (grads_inputs[:,i] for i in range(5))
    g = K.constant(9.81)  # More precise gravity constant
    
    def loss(y_true, y_pred):
        # Data loss
        mse_loss = K.mean(K.square(y_pred - y_true), axis=0)
        data_loss = K.sum(mse_loss)
        
        # Physics loss with improved numerical stability
        u = y_pred[:,0]  # velocity
        h = y_pred[:,1]  # water level
        
        # Saint-Venant equation terms
        momentum_eq = (
            du_dt +                                         # ∂u/∂t
            u * du_dx +                                     # u∂u/∂x
            g * dh_dx +                                     # g∂h/∂x
            g * slope +                                     # gS₀
            g * K.square(fric_coeff) * K.square(u) /       # gn²u²/h^(4/3)
            (K.pow(K.maximum(h, K.epsilon()), 4/3) + K.epsilon())
        )
        
        physics_loss = K.mean(K.square(momentum_eq))
        
        # Combined loss with weighting
        return data_loss + physics_weight * physics_loss
    
    return loss

def create_pinn_model(input_dim, n1, n2, n3, reg_const):
    """Create PINN model with improved architecture"""
    model = Sequential([
        Dense(n1, activation='tanh', kernel_regularizer=l2(reg_const), 
              input_shape=(input_dim,)),
        Dense(n2, activation='tanh', kernel_regularizer=l2(reg_const)),
        Dense(n3, activation='tanh', kernel_regularizer=l2(reg_const)),
        Dense(2, activation='softplus')
    ])

    return model

def train_pinn(X_train, Y_train, X_val, Y_val, n1=64, n2=32, n3=16, 
               reg_const=1e-4, physics_weight=1.0, epochs=100, 
               batch_size=32, patience=5):
    """Train PINN with improved training process"""
    
    '''
    # Normalize data
    (X_train_norm, X_val_norm, Y_train_norm, Y_val_norm,
     X_mean, X_std, Y_mean, Y_std) = normalize_data(X_train, X_val, Y_train, Y_val)
    '''
    # Create and compile model
    model = create_pinn_model(X_train.shape[1], n1, n2, n3, reg_const)
    
    # Calculate gradients for physics loss
    grads_u = K.gradients(model.output[:,0], model.input)[0]
    grads_h = K.gradients(model.output[:,1], model.input)[0]
    du_dx, du_dt, dh_dx = grads_u[:,0], grads_u[:,1], grads_h[:,0]
    calc_grads_inputs = K.stack(
        (du_dx, du_dt, dh_dx, model.input[:,3], model.input[:,4]), 
        axis=1
    )
    
    # Compile with custom loss
    model.compile(
        optimizer='adam',
        loss=custom_loss(calc_grads_inputs, physics_weight),
        metrics=['mape', 'mae', 'mse', NSE, r_square]
    )
    
    # Train with early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=patience,
        restore_best_weights=True
    )
    
    history = model.fit(
        X_train, Y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, Y_val),
        callbacks=[early_stopping],
        verbose=1
    )
    return model, history

In [36]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
'''
def normalize_data(X_train, X_val, Y_train, Y_val):
    """Normalize input and output data"""
    X_mean = np.mean(X_train, axis=0)
    X_std = np.std(X_train, axis=0)
    Y_mean = np.mean(Y_train, axis=0)
    Y_std = np.std(Y_train, axis=0)
    
    X_train_norm = (X_train - X_mean) / (X_std + 1e-8)
    X_val_norm = (X_val - X_mean) / (X_std + 1e-8)
    Y_train_norm = (Y_train - Y_mean) / (Y_std + 1e-8)
    Y_val_norm = (Y_val - Y_mean) / (Y_std + 1e-8)
    
    return (X_train_norm, X_val_norm, Y_train_norm, Y_val_norm, 
            X_mean, X_std, Y_mean, Y_std)
'''
def custom_loss(grads_inputs, physics_weight=1.0):
    """Modified loss function with adjustable physics weight and improved numerical stability"""
    du_dx, du_dt, dh_dx, fric_coeff, slope = (grads_inputs[:,i] for i in range(5))
    g = K.constant(9.81)  # More precise gravity constant
    
    def loss(y_true, y_pred):
        # Data loss
        mse_loss = K.mean(K.square(y_pred - y_true), axis=0)
        data_loss = K.sum(mse_loss)
        
        # Physics loss with improved numerical stability
        u = y_pred[:,0]  # velocity
        h = y_pred[:,1]  # water level
        
        # Saint-Venant equation terms
        momentum_eq = (
            du_dt +                                         # ∂u/∂t
            u * du_dx +                                     # u∂u/∂x
            g * dh_dx +                                     # g∂h/∂x
            g * slope +                                     # gS₀
            g * K.square(fric_coeff) * K.square(u) /       # gn²u²/h^(4/3)
            (K.pow(K.maximum(h, K.epsilon()), 4/3) + K.epsilon())
        )
        
        physics_loss = K.mean(K.square(momentum_eq))
        
        # Combined loss with weighting
        return data_loss + physics_weight * physics_loss
    
    return loss

def create_pinn_model(input_dim, n1, n2, n3, reg_const):
    """Create PINN model with improved architecture"""
    model = Sequential([
        Dense(n1, activation='tanh', kernel_regularizer=l2(reg_const), 
              input_shape=(input_dim,)),
        Dense(n2, activation='tanh', kernel_regularizer=l2(reg_const)),
        Dense(n3, activation='tanh', kernel_regularizer=l2(reg_const)),
        Dense(2, activation='softplus')
    ])

    return model

def train_pinn(X_train, Y_train, X_val, Y_val, n1=64, n2=32, n3=16, 
               reg_const=1e-4, physics_weight=1.0, epochs=100, 
               batch_size=32, patience=5):
    """Train PINN with improved training process"""
    
    '''
    # Normalize data
    (X_train_norm, X_val_norm, Y_train_norm, Y_val_norm,
     X_mean, X_std, Y_mean, Y_std) = normalize_data(X_train, X_val, Y_train, Y_val)
    '''
    # Create and compile model
    model = create_pinn_model(X_train.shape[1], n1, n2, n3, reg_const)
    
    # Calculate gradients for physics loss
    grads_u = K.gradients(model.output[:,0], model.input)[0]
    grads_h = K.gradients(model.output[:,1], model.input)[0]
    du_dx, du_dt, dh_dx = grads_u[:,0], grads_u[:,1], grads_h[:,0]
    calc_grads_inputs = K.stack(
        (du_dx, du_dt, dh_dx, model.input[:,3], model.input[:,4]), 
        axis=1
    )
    
    # Compile with custom loss
    model.compile(
        optimizer='adam',
        loss=custom_loss(calc_grads_inputs, physics_weight),
        metrics=['mape', 'mae', 'mse', NSE, r_square]
    )
    
    # Train with early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=patience,
        restore_best_weights=True
    )
    
    history = model.fit(
        X_train, Y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, Y_val),
        callbacks=[early_stopping],
        verbose=1
    )
    return model, history

In [39]:
# Example usage
model, history = train_pinn(
    X_train, Y_train, X_val, Y_val,
    n1=64, n2=32, n3=16,
    physics_weight=1.0,  
    reg_const=1e-4,
    epochs=100,
    batch_size=32
)

Train on 17520 samples, validate on 17520 samples
Epoch 1/100
17520/17520 [==============================] - 1s 80us/sample - loss: 314.0204 - mape: 2238.6130 - mae: 4.1578 - mse: 46.4000 - NSE: 0.4180 - r_square: 0.0230 - val_loss: 56.4656 - val_mape: 162.4000 - val_mae: 3.0428 - val_mse: 28.2292 - val_NSE: 0.6502 - val_r_square: 3.4156e-04
Epoch 2/100
17520/17520 [==============================] - 1s 66us/sample - loss: 55.0185 - mape: 2047.1155 - mae: 3.0231 - mse: 27.5057 - NSE: 0.6555 - r_square: 0.0037 - val_loss: 56.2828 - val_mape: 163.6745 - val_mae: 3.0355 - val_mse: 28.1378 - val_NSE: 0.6511 - val_r_square: 1.7658e-12
Epoch 3/100
17520/17520 [==============================] - 2s 107us/sample - loss: 54.8195 - mape: 2055.7781 - mae: 3.0148 - mse: 27.4062 - NSE: 0.6570 - r_square: 0.0038 - val_loss: 56.1126 - val_mape: 164.4401 - val_mae: 3.0286 - val_mse: 28.0527 - val_NSE: 0.6520 - val_r_square: 0.0013
Epoch 4/100
17520/17520 [==============================] - 1s 69us/sample